In [1]:
Dataset_url = "https://www.kaggle.com/datasets/tongpython/cat-and-dog"

In [2]:
# Upload Kaggle json file
!mkdir ~/.kaggle/
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d tongpython/cat-and-dog

Dataset URL: https://www.kaggle.com/datasets/tongpython/cat-and-dog
License(s): CC0-1.0
 92% 201M/218M [00:01<00:00, 153MB/s]
100% 218M/218M [00:01<00:00, 130MB/s]


In [4]:
!unzip -q cat-and-dog.zip

In [5]:
!ls

cat-and-dog.zip  kaggle.json  sample_data  test_set  training_set


In [6]:
# Import

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm.auto import tqdm
import time

In [7]:
# Define transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images to a consistent size
    transforms.ToTensor(),  # Convert images to PyTorch tensors
])

In [8]:
train_dataset = datasets.ImageFolder("/content/training_set/training_set/"
                                         , transform=transform)

test_dataset = datasets.ImageFolder("/content/test_set/test_set/"
                                         , transform=transform)

In [9]:
print(f"Train data:\n{train_dataset}\nTest data:\n{test_dataset}")

Train data:
Dataset ImageFolder
    Number of datapoints: 8005
    Root location: /content/training_set/training_set/
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )
Test data:
Dataset ImageFolder
    Number of datapoints: 2023
    Root location: /content/test_set/test_set/
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )


In [10]:
# Get class names as a list
class_names = train_dataset.classes
class_names

['cats', 'dogs']

In [11]:
# Can also get class names as a dict
class_dict = train_dataset.class_to_idx
class_dict

{'cats': 0, 'dogs': 1}

In [12]:
# Check the lengths
len(train_dataset), len(test_dataset)

(8005, 2023)

In [13]:
# Define batch size for dataloader
batch_size = 16

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                         shuffle=True)
test_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                         shuffle=False)

In [14]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
device

'cuda'

In [15]:
def train(dataloader, model, loss_fn, optimizer,device):
    size = len(dataloader.dataset)
    model.to(device)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [16]:
def test(dataloader, model, loss_fn, device):
    size = len(dataloader.dataset)
    model.to(device)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [17]:
# Create a convolutional neural network
class CnnModel(nn.Module):
    """
    Model architecture copying TinyVGG from:
    https://poloclub.github.io/cnn-explainer/
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3, # how big is the square that's going over the image?
                      stride=1, # default
                      padding=1),# options = "valid" (no padding) or "same" (output has same shape as input) or int for specific number
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2) # default stride value is same as kernel_size
        )
        self.block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*56*56,
                      out_features=hidden_units*10*10),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units*10*10,
                      out_features=hidden_units*5*5),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units*5*5,
                      out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units,
                      out_features=output_shape),
        )

    def forward(self, x: torch.Tensor):
        x = self.block_1(x)
        # print(x.shape)
        x = self.block_2(x)
        # print(x.shape)
        x = self.classifier(x)
        # print(x.shape)
        return x

torch.manual_seed(23)
model = CnnModel(input_shape=3,
    hidden_units=50,
    output_shape=2)
model

CnnModel(
  (block_1): Sequential(
    (0): Conv2d(3, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(50, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(50, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(50, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=156800, out_features=5000, bias=True)
    (2): ReLU()
    (3): Linear(in_features=5000, out_features=1250, bias=True)
    (4): ReLU()
    (5): Linear(in_features=1250, out_features=50, bias=True)
    (6): ReLU()
    (7): Linear(in_features=50, out_features=2, bias=True)
  )
)

In [20]:
# Setup loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(),
                             lr=0.1)

In [21]:
from tqdm.auto import tqdm
import time
start_time = time.time()


# Train and test model
epochs = 10
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n---------")
    train(train_dataloader, model, loss_fn, optimizer,device)
    test(test_dataloader, model, loss_fn,device)

current_time = time.time()
total = current_time - start_time
print("Done!")
print(f'Training Took: {total/60} minutes!')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0
---------
loss: 0.722576  [   16/ 8005]
loss: 0.656189  [ 1616/ 8005]
loss: 0.701804  [ 3216/ 8005]
loss: 0.692385  [ 4816/ 8005]
loss: 0.681277  [ 6416/ 8005]
loss: 0.710872  [ 2505/ 8005]
Test Error: 
 Accuracy: 50.0%, Avg loss: 0.692899 

Epoch: 1
---------
loss: 0.692049  [   16/ 8005]
loss: 0.693737  [ 1616/ 8005]
loss: 0.681106  [ 3216/ 8005]
loss: 0.677982  [ 4816/ 8005]
loss: 0.625413  [ 6416/ 8005]
loss: 0.584397  [ 2505/ 8005]
Test Error: 
 Accuracy: 52.4%, Avg loss: 0.857881 

Epoch: 2
---------
loss: 0.678249  [   16/ 8005]
loss: 0.569097  [ 1616/ 8005]
loss: 0.645868  [ 3216/ 8005]
loss: 0.638648  [ 4816/ 8005]
loss: 0.633703  [ 6416/ 8005]
loss: 0.638521  [ 2505/ 8005]
Test Error: 
 Accuracy: 54.6%, Avg loss: 0.738339 

Epoch: 3
---------
loss: 0.567963  [   16/ 8005]
loss: 0.711774  [ 1616/ 8005]
loss: 0.536293  [ 3216/ 8005]
loss: 0.739800  [ 4816/ 8005]
loss: 0.597091  [ 6416/ 8005]
loss: 0.730390  [ 2505/ 8005]
Test Error: 
 Accuracy: 63.1%, Avg loss: 0.65965